In [1]:
import tensorflow as tf
tf.config.set_visible_devices([], device_type='GPU')

2024-10-28 09:19:03.071370: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-28 09:19:08.974298: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
from tqdm.auto import tqdm

from typing import Any, Callable, Sequence, Union
import numpy as np
import scipy.stats as stats

import tensorflow as tf
tf.config.set_visible_devices([], device_type='GPU')

import jax
from jax import lax, random, numpy as jnp
import flax
from flax.core import freeze, unfreeze, FrozenDict
from flax import linen as nn
from flax import struct
from flax.training import train_state
from flax.training import orbax_utils

import optax
import orbax.checkpoint

from clu import metrics
from ml_collections import ConfigDict

from einops import reduce, rearrange
import wandb

from iqadatasets.datasets import *
from fxlayers.layers import *
from fxlayers.layers import GaborLayerLogSigma_, GaussianLayerGamma, FreqGaussianGamma, OrientGaussianGamma
from fxlayers.initializers import *
from JaxPlayground.utils.constraints import *
from JaxPlayground.utils.wandb import *

2024-10-28 09:19:28.465458: W external/xla/xla/service/platform_util.cc:198] unable to create StreamExecutor for CUDA:1: failed initializing StreamExecutor for CUDA device ordinal 1: INTERNAL: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_DEVICE_UNAVAILABLE: CUDA-capable device(s) is/are busy or unavailable
2024-10-28 09:19:28.465510: W external/xla/xla/service/platform_util.cc:198] unable to create StreamExecutor for CUDA:0: failed initializing StreamExecutor for CUDA device ordinal 0: INTERNAL: failed call to cuDevicePrimaryCtxRetain: CUDA_ERROR_DEVICE_UNAVAILABLE: CUDA-capable device(s) is/are busy or unavailable
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [3]:
# dst_train = TID2008("/lustre/ific.uv.es/ml/uv075/Databases/IQA//TID/TID2008/", exclude_imgs=[25])
# dst_val = TID2013("/lustre/ific.uv.es/ml/uv075/Databases/IQA//TID/TID2013/", exclude_imgs=[25])
# dst_train = TID2008("/media/disk/databases/BBDD_video_image/Image_Quality//TID/TID2008/", exclude_imgs=[25])
# dst_val = TID2013("/media/disk/databases/BBDD_video_image/Image_Quality//TID/TID2013/", exclude_imgs=[25])
# dst = KADIK10K("/media/disk/databases/BBDD_video_image/Image_Quality/KADIK10K/")
# dst = PIPAL("/media/disk/databases/BBDD_video_image/Image_Quality/PIPAL/")
dst = TID2008("/lustre/ific.uv.es/ml/uv075/Databases/IQA//TID/TID2008/")

In [4]:
img, img_dist, mos = next(iter(dst.dataset))
img.shape, img_dist.shape, mos.shape

2024-10-28 09:19:29.045045: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype double and shape [1700]
	 [[{{node Placeholder/_2}}]]


(TensorShape([384, 512, 3]), TensorShape([384, 512, 3]), TensorShape([]))

In [24]:
import torch
from piq import LPIPS

In [34]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device = None
device

In [36]:
model = LPIPS(reduction="none")
model = model.to(device)

In [11]:
config = ConfigDict({
    "BATCH_SIZE": 64,
})

In [12]:
wandb.init(project="PerceptNet_JaX_Eval",
           name="LPIPS",
           job_type="evaluate",
           mode="disabled",
           )
config

BATCH_SIZE: 64

In [13]:
dst_rdy = dst.dataset.batch(config.BATCH_SIZE, num_parallel_calls=tf.data.AUTOTUNE)

In [14]:
for batch in dst_rdy.as_numpy_iterator():
    break

2024-10-28 09:32:13.322765: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1700]
	 [[{{node Placeholder/_0}}]]
2024-10-28 09:32:13.323537: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_2' with dtype double and shape [1700]
	 [[{{node Placeholder/_2}}]]


In [45]:
def compute_distance(model, batch, device=None):
    img, img_dist, mos = batch
    img = img.transpose(0,3,1,2)
    img_dist = img_dist.transpose(0,3,1,2)
    img, img_dist = torch.Tensor(img), torch.Tensor(img_dist)
    img, img_dist = img.to(device), img_dist.to(device)

    with torch.no_grad():
        dist = model(img, img_dist)

    return dist

## Evaluate!

In [46]:
metrics_history = {
    "distance": [],
    "mos": [],
}

In [47]:
%%time
for batch in tqdm(dst_rdy.as_numpy_iterator()):
    img, img_dist, mos = batch
    distance = compute_distance(model=model, batch=batch, device=None)
    metrics_history["distance"].extend(distance.numpy())
    metrics_history["mos"].extend(mos)
    break

0it [00:00, ?it/s]

CPU times: user 11min 6s, sys: 4min 22s, total: 15min 28s
Wall time: 37 s


In [ ]:
assert len(metrics_history["distance"]) == len(dst.data)

In [50]:
stats.pearsonr(metrics_history["distance"], metrics_history["mos"]), stats.spearmanr(metrics_history["distance"], metrics_history["mos"])

(PearsonRResult(statistic=-0.7809211408422801, pvalue=2.7273724860032935e-14),
 SignificanceResult(statistic=-0.694438722246202, pvalue=1.9642521293679142e-10))

In [6]:
results = dst.data.copy()
results["Distance"] = metrics_history["distance"]
results.head()

,Reference,Distorted,MOS,Reference_ID,Distortion_ID,Distortion_Intensity,Distance
0,I01.BMP,I01_01_1.bmp,5.9706,1,1,1,71.772995
1,I01.BMP,I01_01_2.bmp,5.4167,1,1,2,101.15051
2,I01.BMP,I01_01_3.bmp,4.5556,1,1,3,130.07452
3,I01.BMP,I01_01_4.bmp,4.3143,1,1,4,159.60564
4,I01.BMP,I01_02_1.bmp,6.1429,1,2,1,54.716625


In [ ]:
wandb.log({"TID2008": wandb.Table(dataframe=results),
           "TID2008_pearson": stats.pearsonr(metrics_history["distance"], metrics_history["mos"])[0],
           "TID2008_spearman": stats.spearmanr(metrics_history["distance"], metrics_history["mos"])[0],
           })

<class 'wandb.errors.Error'>: You must call wandb.init() before wandb.log()

In [ ]:
wandb.finish()